In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from pylab import rcParams
from IPython.display import display
import math
pd.set_option('display.max_columns', 130)

import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
# pd.set_option('future.no_silent_downcasting', False)


#### Data preparation

In [2]:
# open and read excel file
df = pd.read_excel("../input data/Данные-рус Обработанные (новые).xlsx")
df.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,2015-08-01 00:00:00,2015-09-01 00:00:00,2015-10-01 00:00:00,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,2016-10-01 00:00:00,2016-11-01 00:00:00,2016-12-01 00:00:00,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,2017-09-01 00:00:00,2017-10-01 00:00:00,2017-11-01 00:00:00,2017-12-01 00:00:00,2018-01-01 00:00:00,2018-02-01 00:00:00,2018-03-01 00:00:00,2018-04-01 00:00:00,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-07-01 00:00:00,2018-08-01 00:00:00,2018-09-01 00:00:00,2018-10-01 00:00:00,2018-11-01 00:00:00,2018-12-01 00:00:00,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00,2020-04-01 00:00:00,2020-05-01 00:00:00,2020-06-01 00:00:00,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,2021-09-01 00:00:00,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-05-01 00:00:00,2022-06-01 00:00:00,2022-07-01 00:00:00,2022-08-01 00:00:00,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00

In [3]:
# remove the first row (blank)
df = df.drop(index=df.index[:1]).reset_index(drop=True)
df.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122
0,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,2015-08-01 00:00:00,2015-09-01 00:00:00,2015-10-01 00:00:00,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,2016-10-01 00:00:00,2016-11-01 00:00:00,2016-12-01 00:00:00,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,2017-09-01 00:00:00,2017-10-01 00:00:00,2017-11-01 00:00:00,2017-12-01 00:00:00,2018-01-01 00:00:00,2018-02-01 00:00:00,2018-03-01 00:00:00,2018-04-01 00:00:00,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-07-01 00:00:00,2018-08-01 00:00:00,2018-09-01 00:00:00,2018-10-01 00:00:00,2018-11-01 00:00:00,2018-12-01 00:00:00,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00,2020-04-01 00:00:00,2020-05-01 00:00:00,2020-06-01 00:00:00,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,2021-09-01 00:00:00,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-05-01 00:00:00,2022-06-01 00:00:00,2022-07-01 00:00:00,2022-08-01 00:00:00,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00,2024-01-01 00:00:00,2024-02-01 00:00:00,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
1,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5

In [4]:
# Set the first row as column labels
df.columns = df.iloc[0]  # Assign first row as header
df = df[1:].reset_index(drop=True)  # Drop the first row and reset index
df.head(5)

,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,2015-08-01 00:00:00,2015-09-01 00:00:00,2015-10-01 00:00:00,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,2016-10-01 00:00:00,2016-11-01 00:00:00,2016-12-01 00:00:00,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,2017-04-01 00:00:00,2017-05-01 00:00:00,2017-06-01 00:00:00,2017-07-01 00:00:00,2017-08-01 00:00:00,2017-09-01 00:00:00,2017-10-01 00:00:00,2017-11-01 00:00:00,2017-12-01 00:00:00,2018-01-01 00:00:00,2018-02-01 00:00:00,2018-03-01 00:00:00,2018-04-01 00:00:00,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-07-01 00:00:00,2018-08-01 00:00:00,2018-09-01 00:00:00,2018-10-01 00:00:00,2018-11-01 00:00:00,2018-12-01 00:00:00,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00,2020-04-01 00:00:00,2020-05-01 00:00:00,2020-06-01 00:00:00,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,2021-09-01 00:00:00,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-05-01 00:00:00,2022-06-01 00:00:00,2022-07-01 00:00:00,2022-08-01 00:00:00,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00,2024-01-01 00:00:00,2024-02-01 00:00:00,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
0,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,4014.95,4907.52,4274.75,3858.39,6477.35,3884.84,4146.9,4282.08,5408.9,4390.56,8005.48,5105.84,4700.12,5548.88,3958.09,6264.34,9196.32,3739.75,4951.17,6316.68,5751.36,5747.16,10114.01,5944.43,6768.61,5781.59,5885.5,7429.34,9411.91,5555.9,4586.48,7970.28,8130.34,8048.48,14491.87,7543.26,7396.97,9668.6,6730.17,7987.91,11121.61,5666.49,5917.68,8899.55,7784.95,9167.13,16299.73,7164.2,7341.17,10162.65,13532.56,11737.17,14948.7,7041.38,4341.74,10336.12,7419.05,10452.61,16413.66,7973.85,8901.52,10934.83,13130.82,12848.98,18882.81,8135.62,6432.18,9650.38,10482.45,11405.76,16121.49,9114.93,14885.29,13567.02,13473.97,12558.44,20382.82,6709.4,8554.71,12487.58,9184.25,7094.7,16471.11,6373.02,10145.08,8831.12,11123,8927.05,11379.06,7452.75,6836.85,8719.53,9925.26,7684.37,12218.39,6460.2,11289.58,15156.55,9790.74,9021.76,13193.63,6799.94,12303.66,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
1,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,942.18,856.9,981.04,873.89,1390.18,736.62,845.64,774.36,618.97,701.72,796.48,914.5,736.52,897.89,912.46,867.99,1102.48,660.12,731.14,835,860.71,883.43,963.32,745.3,844.01,678,1025.12,1312.34,919.29,670.25,942.66,847.27,907.92,1002.24,911.62,873.09,1070.9,938.24,832.71,780.14,792.74,993.76,1044.12,1250.31,892.18,1441.82,1249.59,822.67,1007.64,1333,1160.5,1088.61,1164.95,782.49,829.6,1780.71,1077.39,1145.56,1100.69,898.78,829.67,930.63

In [5]:
# convert date columns to datetime format
df.columns = list(df.columns[:3]) + list(pd.to_datetime(df.columns[-120:]))
df = df.rename(columns={col: col.strftime("%Y-%m") for col in df.columns[-120:]}) # change "2015-01-01 00:00:00" to "2015-01" 
df.head(5)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,4014.95,4907.52,4274.75,3858.39,6477.35,3884.84,4146.9,4282.08,5408.9,4390.56,8005.48,5105.84,4700.12,5548.88,3958.09,6264.34,9196.32,3739.75,4951.17,6316.68,5751.36,5747.16,10114.01,5944.43,6768.61,5781.59,5885.5,7429.34,9411.91,5555.9,4586.48,7970.28,8130.34,8048.48,14491.87,7543.26,7396.97,9668.6,6730.17,7987.91,11121.61,5666.49,5917.68,8899.55,7784.95,9167.13,16299.73,7164.2,7341.17,10162.65,13532.56,11737.17,14948.7,7041.38,4341.74,10336.12,7419.05,10452.61,16413.66,7973.85,8901.52,10934.83,13130.82,12848.98,18882.81,8135.62,6432.18,9650.38,10482.45,11405.76,16121.49,9114.93,14885.29,13567.02,13473.97,12558.44,20382.82,6709.4,8554.71,12487.58,9184.25,7094.7,16471.11,6373.02,10145.08,8831.12,11123,8927.05,11379.06,7452.75,6836.85,8719.53,9925.26,7684.37,12218.39,6460.2,11289.58,15156.55,9790.74,9021.76,13193.63,6799.94,12303.66,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
1,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,942.18,856.9,981.04,873.89,1390.18,736.62,845.64,774.36,618.97,701.72,796.48,914.5,736.52,897.89,912.46,867.99,1102.48,660.12,731.14,835,860.71,883.43,963.32,745.3,844.01,678,1025.12,1312.34,919.29,670.25,942.66,847.27,907.92,1002.24,911.62,873.09,1070.9,938.24,832.71,780.14,792.74,993.76,1044.12,1250.31,892.18,1441.82,1249.59,822.67,1007.64,1333,1160.5,1088.61,1164.95,782.49,829.6,1780.71,1077.39,1145.56,1100.69,898.78,829.67,930.63,1135.02,1025.99,1133.29,1141.02,957.47,1261.8,979.49,1029.19,864.06,1056.22,1019.37,1028.55,1254.05,1025.9,1062.94,537.96,1089,1631.34,1183.73,1118.31,1064.21,1240.11,1000.26,1269.72,1136.14,1230.95,1026.95,1294.11,936.76,1229.11,1048.76,1040.48,1181.82,869.84,993.64,938.85,1102.67,1001.63,1079.39,972.07,938.35,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
2,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,303.09,685.51,600.85,603.26,700.48,477.84,221.08,265.59,328.03,384.03,555.07,615.9,619.33,770.79,302.27,1076.31,862.54,473.66,475.3,814.18,638.87,642.96,834.97,902.65,701.17,756.63,726.68,654.31,801.49,703.32,514.47,952.16,1018.39,968.68,845.85,758.18,931.47,1070.07,1278.41,1627.73,1714.64,704.51,899.66,1554.1,1351.44,1050.71,1485.45,814.13,1169.81,2388.34,1595.4,3809.76,3510.19,954.53,481.35,1111.55,1033.96,1939.87,2152.47,1578.87,1426.39,2620.87,2434.4,3247.3,3281.67,580.89,823.34,1221.21,1626.06,1920.41,2637.56,1620.81,2134.96,3425.67,3449.02,3751.75,3827.39,826.17,1132.81,1544.91,1885.46,698.01,4712.69,508.07,592.19,902.26,1479.36,1219.25,658.78,589.8,1275.24,1508.27,1380.18,842.24,3478.29,774.46,764.8,3003.94,1876.68,1547.77,607.03,609.08,873.05,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69
3,КРС,Сельхозпредприятия,АЛМАТИНСКАЯ ОБЛАСТЬ,178.3,362.86,352.49,451.79,228.32,708.84,280.55,126.59,629.44,314.28,302.7,

In [6]:
# check values
print(df["Показатель"].unique())
print(df["Категория хозяйства"].unique())
print(df["Регион"].unique())

['КРС' 'Свиньи' 'Лошади' 'Верблюды' 'Птица' 'Молоко' 'Яйца' 'Овцы и козы']
['Сельхозпредприятия' 'Крестьянские или фермерские хозяйства'
 'Хозяйства населения']
['РЕСПУБЛИКА КАЗАХСТАН' '    АКМОЛИНСКАЯ ОБЛАСТЬ'
 '    АКТЮБИНСКАЯ ОБЛАСТЬ' '    АЛМАТИНСКАЯ ОБЛАСТЬ'
 '    АТЫРАУСКАЯ ОБЛАСТЬ' '    ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ЖАМБЫЛСКАЯ ОБЛАСТЬ' '    КАРАГАНДИНСКАЯ ОБЛАСТЬ'
 '    КОСТАНАЙСКАЯ ОБЛАСТЬ' '    КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ'
 '    МАНГИСТАУСКАЯ ОБЛАСТЬ' '    ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ПАВЛОДАРСКАЯ ОБЛАСТЬ' '    СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ' '    ГАСТАНА' '    ГАЛМАТЫ'
 '    ГШЫМКЕНТ' '    ТУРКЕСТАНСКАЯ ОБЛАСТЬ' '    ОБЛАСТЬ ЖЕТІСУ'
 '    ОБЛАСТЬ ҰЛЫТАУ' '    ОБЛАСТЬ АБАЙ']


In [7]:
df["Регион"] = df["Регион"].str.strip() # strip unnecessary spaces
df = df[df["Регион"] != "ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ"] # remove ЮКО
df["Регион"].unique()

array(['РЕСПУБЛИКА КАЗАХСТАН', 'АКМОЛИНСКАЯ ОБЛАСТЬ',
       'АКТЮБИНСКАЯ ОБЛАСТЬ', 'АЛМАТИНСКАЯ ОБЛАСТЬ', 'АТЫРАУСКАЯ ОБЛАСТЬ',
       'ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ЖАМБЫЛСКАЯ ОБЛАСТЬ',
       'КАРАГАНДИНСКАЯ ОБЛАСТЬ', 'КОСТАНАЙСКАЯ ОБЛАСТЬ',
       'КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ', 'МАНГИСТАУСКАЯ ОБЛАСТЬ',
       'ПАВЛОДАРСКАЯ ОБЛАСТЬ', 'СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ',
       'ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ГАСТАНА', 'ГАЛМАТЫ', 'ГШЫМКЕНТ',
       'ТУРКЕСТАНСКАЯ ОБЛАСТЬ', 'ОБЛАСТЬ ЖЕТІСУ', 'ОБЛАСТЬ ҰЛЫТАУ',
       'ОБЛАСТЬ АБАЙ'], dtype=object)

In [8]:
# fill missing values with 0
df = df.fillna(0)
# replace "-" with 0
df.replace("-", 0, inplace=True)
df.tail(5)

C:\Users\Acer\AppData\Local\Temp\ipykernel_19196\4275439588.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("-", 0, inplace=True)


,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
523,Овцы и козы,Хозяйства населения,ГШЫМКЕНТ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,113.19,109.50,107.50,109.61,110.10,123.3,329.00,91.10,80.10,84.1,84.50,90.90,82.30,88.80,74.00,75.10,84.40,78.30,339.50,87.8,86.2,84.1,84.5,90.9,82.3,88.8,74.0,75.1,84.4,78.3,75.1,88.3,70.4,56.5,45.6,50.6,46.2,42.7,40.7,47.8,50.6,47.9,359.5,89.1,70.7,56.7,44.7,46.4,46.3,42.0,39.7,46.3,48.4,46.2,174.8,88.30,68.30,54.70,43.50,46.40,44.20,42.10,39.70,46.30,48.40,46.20,174.80,88.2,68.3,54.6,43.7,46.4,44.2,42.0,39.7,50.7,49.0,46.2,174.8
524,Овцы и козы,Хозяйства населения,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5199.07,4570.03,4854.96,5303.53,6090.55,5718.8,7403.46,6204.08,5733.43,5593.5,4994.46,4774.99,4728.19,4398.32,4936.91,5536.35,6242.61,5720.59,7478.67,6347.3,5715.5,5566.5,4964.1,4664.4,4818.6,4424.8,4843.6,5517.7,6199.0,5702.3,7601.6,6457.0,5803.5,5615.4,4981.1,4699.8,4817.4,4410.3,4799.2,5529.5,6247.3,5730.3,7829.9,6525.4,5870.5,5506.0,4898.4,4647.5,4709.3,4163.5,4586.1,5378.3,5998.9,5568.9,7787.0,6503.13,5814.36,5337.86,4678.92,4479.58,4572.33,3427.03,3947.03,5202.49,5752.46,5381.07,7348.42,4421.7,4445.2,3904.3,3414.9,3224.1,3182.1,1823.1,2272.9,3341.6,3790.5,4074.5,5796.7
525,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ЖЕТІСУ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3358.6,591.2,1237.6,2046.3,2415.8,2173.3,3951.1,459.09,472.31,2054.34,645.26,493.88,2984.48,527.36,1130.62,2069.94,2129.18,2175.72,3669.04,464.5,477.4,2077.3,656.4,500.8,3064.5,547.4,1156.3,2134.4,2237.4,2093.4,3738.4
526,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ҰЛЫТАУ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281.8,87.6,137.0,455.0,182.9,184.8,634.3,188.43,84.88,239.96,64.64,489.30,422.24,85.07,207.07,657.05,218.33,214.88,217.80,162.1,220.6,198.3,65.4,490.1,376.7,86.4,213.8,455.5,216.7,169.2,216.1
527,Овцы и козы,Хозяйства населения,ОБЛАСТЬ АБАЙ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.

In [9]:
# Step 1: Define target regions and periods
regions_to_null = ['ОБЛАСТЬ ЖЕТІСУ', 'ОБЛАСТЬ ҰЛЫТАУ', 'ОБЛАСТЬ АБАЙ']
periods_to_null = pd.date_range("2015-01", "2022-05", freq="MS").strftime("%Y-%m")

# Step 2: Replace matching values with NaN
mask = (
    df["Регион"].isin(regions_to_null) &
    df.columns[3:].to_series().isin(periods_to_null).any()
)

# Step 3: Set values to NaN across all columns for those rows and columns
value_columns = df.columns[3:]  # assuming first 3 columns are identifiers

for period in periods_to_null:
    if period in value_columns:
        df.loc[df["Регион"].isin(regions_to_null), period] = np.nan
df.tail(5)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
523,Овцы и козы,Хозяйства населения,ГШЫМКЕНТ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,113.19,109.50,107.50,109.61,110.10,123.3,329.00,91.10,80.10,84.1,84.50,90.90,82.30,88.80,74.00,75.10,84.40,78.30,339.50,87.8,86.2,84.1,84.5,90.9,82.3,88.8,74.0,75.1,84.4,78.3,75.1,88.3,70.4,56.5,45.6,50.6,46.2,42.7,40.7,47.8,50.6,47.9,359.5,89.1,70.7,56.7,44.7,46.4,46.3,42.0,39.7,46.3,48.4,46.2,174.8,88.30,68.30,54.70,43.50,46.40,44.20,42.10,39.70,46.30,48.40,46.20,174.80,88.2,68.3,54.6,43.7,46.4,44.2,42.0,39.7,50.7,49.0,46.2,174.8
524,Овцы и козы,Хозяйства населения,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5199.07,4570.03,4854.96,5303.53,6090.55,5718.8,7403.46,6204.08,5733.43,5593.5,4994.46,4774.99,4728.19,4398.32,4936.91,5536.35,6242.61,5720.59,7478.67,6347.3,5715.5,5566.5,4964.1,4664.4,4818.6,4424.8,4843.6,5517.7,6199.0,5702.3,7601.6,6457.0,5803.5,5615.4,4981.1,4699.8,4817.4,4410.3,4799.2,5529.5,6247.3,5730.3,7829.9,6525.4,5870.5,5506.0,4898.4,4647.5,4709.3,4163.5,4586.1,5378.3,5998.9,5568.9,7787.0,6503.13,5814.36,5337.86,4678.92,4479.58,4572.33,3427.03,3947.03,5202.49,5752.46,5381.07,7348.42,4421.7,4445.2,3904.3,3414.9,3224.1,3182.1,1823.1,2272.9,3341.6,3790.5,4074.5,5796.7
525,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ЖЕТІСУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3358.6,591.2,1237.6,2046.3,2415.8,2173.3,3951.1,459.09,472.31,2054.34,645.26,493.88,2984.48,527.36,1130.62,2069.94,2129.18,2175.72,3669.04,464.5,477.4,2077.3,656.4,500.8,3064.5,547.4,1156.3,2134.4,2237.4,2093.4,3738.4
526,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ҰЛЫТАУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,281.8,87.6,137.0,455.0,182.9,184.8,634.3,188.43,84.88,239.96,64.64,489.30,422.24,85.07,207.07,657.05,218.33,214.88,217.80,162.1,220.6,198.3,65.4,490.1,376.7,86.4,213.8,455.5,216.7,169.2,216.1
527,Овцы и козы,Хозяйства населения,ОБЛАСТЬ АБАЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [10]:
# Step 1: Define target regions and periods
regions_to_null2 = ['ГШЫМКЕНТ', 'ТУРКЕСТАНСКАЯ ОБЛАСТЬ']
periods_to_null2 = pd.date_range("2015-01", "2018-05", freq="MS").strftime("%Y-%m")

# Step 2: Replace matching values with NaN
mask = (
    df["Регион"].isin(regions_to_null2) &
    df.columns[3:].to_series().isin(periods_to_null2).any()
)

# Step 3: Set values to NaN across all columns for those rows and columns
value_columns = df.columns[3:]  # assuming first 3 columns are identifiers

for period in periods_to_null2:
    if period in value_columns:
        df.loc[df["Регион"].isin(regions_to_null2), period] = np.nan
df.tail(5)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
523,Овцы и козы,Хозяйства населения,ГШЫМКЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.19,109.50,107.50,109.61,110.10,123.3,329.00,91.10,80.10,84.1,84.50,90.90,82.30,88.80,74.00,75.10,84.40,78.30,339.50,87.8,86.2,84.1,84.5,90.9,82.3,88.8,74.0,75.1,84.4,78.3,75.1,88.3,70.4,56.5,45.6,50.6,46.2,42.7,40.7,47.8,50.6,47.9,359.5,89.1,70.7,56.7,44.7,46.4,46.3,42.0,39.7,46.3,48.4,46.2,174.8,88.30,68.30,54.70,43.50,46.40,44.20,42.10,39.70,46.30,48.40,46.20,174.80,88.2,68.3,54.6,43.7,46.4,44.2,42.0,39.7,50.7,49.0,46.2,174.8
524,Овцы и козы,Хозяйства населения,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5199.07,4570.03,4854.96,5303.53,6090.55,5718.8,7403.46,6204.08,5733.43,5593.5,4994.46,4774.99,4728.19,4398.32,4936.91,5536.35,6242.61,5720.59,7478.67,6347.3,5715.5,5566.5,4964.1,4664.4,4818.6,4424.8,4843.6,5517.7,6199.0,5702.3,7601.6,6457.0,5803.5,5615.4,4981.1,4699.8,4817.4,4410.3,4799.2,5529.5,6247.3,5730.3,7829.9,6525.4,5870.5,5506.0,4898.4,4647.5,4709.3,4163.5,4586.1,5378.3,5998.9,5568.9,7787.0,6503.13,5814.36,5337.86,4678.92,4479.58,4572.33,3427.03,3947.03,5202.49,5752.46,5381.07,7348.42,4421.7,4445.2,3904.3,3414.9,3224.1,3182.1,1823.1,2272.9,3341.6,3790.5,4074.5,5796.7
525,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ЖЕТІСУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3358.6,591.2,1237.6,2046.3,2415.8,2173.3,3951.1,459.09,472.31,2054.34,645.26,493.88,2984.48,527.36,1130.62,2069.94,2129.18,2175.72,3669.04,464.5,477.4,2077.3,656.4,500.8,3064.5,547.4,1156.3,2134.4,2237.4,2093.4,3738.4
526,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ҰЛЫТАУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,281.8,87.6,137.0,455.0,182.9,184.8,634.3,188.43,84.88,239.96,64.64,489.30,422.24,85.07,207.07,657.05,218.33,214.88,217.80,162.1,220.6,198.3,65.4,490.1,376.7,86.4,213.8,455.5,216.7,169.2,216.1
527,Овцы и козы,Хозяйства населения,ОБЛАСТЬ АБАЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [11]:
# check replacement
df.groupby("Регион")[df.columns[90:94]].apply(lambda g: g.isna().sum()).reset_index()

,Регион,2022-04,2022-05,2022-06,2022-07
0,АКМОЛИНСКАЯ ОБЛАСТЬ,0,0,0,0
1,АКТЮБИНСКАЯ ОБЛАСТЬ,0,0,0,0
2,АЛМАТИНСКАЯ ОБЛАСТЬ,0,0,0,0
3,АТЫРАУСКАЯ ОБЛАСТЬ,0,0,0,0
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0,0,0,0
5,ГАЛМАТЫ,0,0,0,0
6,ГАСТАНА,0,0,0,0
7,ГШЫМКЕНТ,0,0,0,0
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,0,0,0,0
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0,0,0,0


### 

In [12]:
df.groupby("Регион")[df.columns[42:46]].apply(lambda g: g.isna().sum()).reset_index()

,Регион,2018-04,2018-05,2018-06,2018-07
0,АКМОЛИНСКАЯ ОБЛАСТЬ,0,0,0,0
1,АКТЮБИНСКАЯ ОБЛАСТЬ,0,0,0,0
2,АЛМАТИНСКАЯ ОБЛАСТЬ,0,0,0,0
3,АТЫРАУСКАЯ ОБЛАСТЬ,0,0,0,0
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0,0,0,0
5,ГАЛМАТЫ,0,0,0,0
6,ГАСТАНА,0,0,0,0
7,ГШЫМКЕНТ,24,24,0,0
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,0,0,0,0
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0,0,0,0


In [13]:
non_numeric = df[df.columns[3:]].select_dtypes(exclude='number').columns
print("Non-numeric columns:", list(non_numeric))
for col in non_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

Non-numeric columns: ['2020-02', '2022-10', '2024-02']


In [14]:
# get the final data by summing across показатели & категория хозяйства
totals = df.groupby(["Показатель", "Регион"])[df.columns[3:]].agg(lambda x: x.sum() if x.notna().any() else np.nan).reset_index()
totals

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,0.00,0.00,0.20,0.00,0.00,0.18,0.28,0.00,0.00,0.40,0.00,0.00,0.65,0.00,0.31,1.00,0.00,0.00,0.00,0.00,0.00,2.01,0.00,1.20,0.00,0.00,2.18,0.39,0.00,0.00,1.04,0.00,0.14,2.08,0.00,0.00,0.00,0.00,0.00,0.30,0.00,0.00,0.00,0.66,0.00,0.33,0.00,0.9,0.00,0.00,0.00,10.08,0.00,0.0,0.00,0.0,0.00,0.54,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,39.16,46.16,238.56,463.35,109.04,72.94,384.96,114.35,221.89,10.93,18.79,3.50,38.53,46.64,216.08,472.07,110.32,68.17,380.15,127.12,217.61,14.98,21.26,7.14,44.63,51.78,239.43,513.33,115.70,69.40,371.53,130.17,218.41,14.59,21.76,7.19,45.97,55.48,252.68,527.66,117.94,70.37,385.75,117.80,218.45,15.54,21.78,7.2,47.06,58.02,257.54,543.35,119.80,71.3,396.90,121.8,228.10,15.90,21.9,7.2,63.00,59.90,260.20,552.10,121.60,71.8,398.40,128.50,228.80,13.10,22.50,7.20,63.90,61.90,150.70,554.90,121.30,73.3,404.20,128.70,234.90,13.60,23.60,6.3,49.80,62.4,153.70,552.8,118.07,69.17,384.21,121.01,223.46,11.76,21.39,5.52,48.18,61.39,150.87,531.79,119.50,72.60,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,16.09,0.10,10.80,46.78,15.06,13.70,131.80,9.20,17.89,130.12,2.00,0.00,4.99,2.06,4.90,44.48,15.06,23.94,41.60,19.54,3.00,15.14,18.50,14.82,14.52,12.60,9.92,43.11,20.72,2.96,15.24,0.00,1.00,20.03,3.44,7.77,117.25,1.00,0.00,20.34,21.95,3.04,17.91,26.81,9.16,18.80,13.37,0.0,17.29,15.90,4.69,23.10,10.85,9.7,42.75,0.0,6.21,21.10,4.0,2.6,28.10,4.53,79.50,93.18,23.70,11.2,16.00,2.00,3.10,19.62,2.30,4.85,11.18,2.80,9.65,36.44,10.60,11.5,26.10,7.20,12.75,11.20,11.20,27.0,25.26,17.9,21.90,12.5,16.55,16.81,22.78,12.10,12.44,6.85,39.65,17.22,8.85,27.48,0.48,29.72,18.90,17.40,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,262.37,193.97,308.17,1087.33,325.10,190.60,301.10,154.84,328.50,220.30,66.10,118.20,234.90,196.51,270.92,974.18,303.30,167.41,323.54,182.34,349.10,249.40,57.57,88.40,221.21,199.63,278.80,964.02,305.22,159.62,326.87,159.50,367.20,257.50,60.01,78.10,253.27,332.00,355.05,989.86,280.08,165.05,334.51,134.85,367.97,321.22,126.22,93.7,263.10,366.66,348.54,1041.38,293.40,154.8,339.44,143.6,405.80,308.35,88.5,130.5,575.30,458.70,357.62,998.10,292.76,188.3,487.02,129.28,421.44,351.20,95.77,97.54,624.80,631.65,499.30,1179.66,286.67,189.8,513.16,171.46,402.10,432.22,121.53,126.2,637.50,651.7,482.94,1412.2,307.85,200.59,466.60,182.63,414.56,405.19,184.80,154.96,717.92,719.01,522.83,1164.77,323.81,423.76,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70

### Concat with other indicators dataset (температура, осадки, поголовье)

In [18]:
x_vars = pd.read_excel("output data/доп регрессоры обработанные.xlsx")
x_vars.head(5)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Температура,АКМОЛИНСКАЯ ОБЛАСТЬ,-12.490323,-10.192857,-5.870968,4.490000,14.574194,20.763333,19.180645,16.529032,11.496667,2.925806,-8.270000,-6.145161,-17.790323,-7.431034,-2.167742,8.633333,13.106452,17.490000,19.106452,20.048387,13.530000,0.370968,-9.976667,-12.954839,-12.264516,-12.989286,-5.403226,5.603333,13.835484,19.546667,18.829032,19.441935,11.596667,2.796774,-1.766667,-10.670968,-18.625806,-14.717857,-8.196774,3.550000,9.506452,17.113333,20.348387,16.616129,11.983333,5.232258,-4.856667,-13.967742,-12.832258,-14.260714,-2.048387,4.333333,13.312903,16.046667,21.670968,18.267742,10.853333,6.922581,-7.266667,-8.038710,-7.838710,-5.851724,-1.441935,9.550000,17.687097,17.110000,21.548387,19.616129,11.306667,5.303226,-5.140000,-13.048387,-17.587097,-14.053571,-7.406452,5.130000,18.716129,18.466667,21.235484,20.074194,10.353333,4.506452,-5.856667,-8.493548,-11.683871,-9.200000,-8.535484,8.873333,15.338710,18.736667,20.583871,17.632258,13.930000,4.709677,-7.810000,-14.154839,-11.600000,-12.278571,-1.867742,5.656667,14.829032,19.796667,24.054839,18.738710,12.866667,6.474194,0.103333,-11.683871,-12.638710,-13.644828,-4.809677,8.026667,10.696774,21.043333,20.270968,17.338710,11.300000,3.832258,-4.343333,-7.638710
1,Температура,АКТЮБИНСКАЯ ОБЛАСТЬ,-13.483871,-11.821429,-5.990323,6.356667,15.609677,25.126667,23.167742,20.732258,16.576667,4.187097,-2.920000,-3.845161,-12.474194,-3.806897,-1.054839,9.783333,16.167742,19.766667,22.332258,24.861290,13.863333,3.980645,-4.056667,-10.980645,-10.119355,-12.132143,-5.500000,5.953333,14.645161,19.843333,23.587097,23.758065,15.550000,5.164516,1.133333,-10.583871,-15.477419,-11.739286,-7.812903,6.746667,16.445161,18.810000,25.893548,20.570968,15.266667,6.838710,-4.470000,-9.922581,-12.296774,-11.685714,-0.583871,8.230000,17.806452,22.266667,24.580645,19.545161,10.923333,8.012903,-4.690000,-7.016129,-5.767742,-5.320690,0.874194,8.100000,17.722581,21.006667,25.261290,20.774194,13.356667,6.409677,-4.913333,-13.645161,-11.145161,-10.978571,-6.683871,7.923333,20.609677,24.920000,23.458065,24.222581,12.753333,5.693548,-3.143333,-6.532258,-9.961290,-5.875000,-6.783871,10.880000,13.245161,20.303333,22.854839,23.264516,15.700000,6.512903,-2.326667,-11.690323,-15.625806,-10.800000,1.522581,10.190000,17.077419,21.763333,24.461290,22.083871,14.703333,7.032258,2.976667,-7.170968,-11.206452,-11.568966,-4.800000,13.513333,13.083871,22.893333,22.912903,20.206452,14.083333,6.667742,-1.520000,-6.419355
2,Температура,АЛМАТИНСКАЯ ОБЛАСТЬ,-5.432258,-1.100000,2.909677,12.126667,17.180645,20.980000,25.564516,22.100000,14.036667,9.780645,2.073333,-3.306452,-2.987097,-3.148276,7.732258,12.073333,15.458065,21.406667,22.287097,21.003226,18.433333,5.293548,-2.963333,-2.283871,-5.312903,-4.157143,2.177419,10.100000,17.093548,20.503333,24.967742,21.006452,16.506667,8.367742,2.550000,-4.222581,-13.251613,-4.892857,6.809677,10.826667,14.519355,20.166667,23.100000,22.412903,14.736667,8.312903,-2.760000,-5.429032,-4.593548,-3.853571,6.267742,11.760000,15.341935,2

In [19]:
x_vars['Показатель'].unique()

array(['Температура', 'Поголовье: КРС', 'Поголовье: лошади',
       'Поголовье: овцы и козы', 'Поголовье: свиньи',
       'Поголовье: птица домашняя', 'Поголовье: верблюды', 'Осадки'],
      dtype=object)

In [20]:
df_combined = pd.concat([totals, x_vars], ignore_index=True)
df_combined.sample(10)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
225,Поголовье: лошади,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,340731.00,313183.00,308685.00,304241.00,297891.00,291493.00,320491.0,316045.00,314708.0,324051.00,348449.00,357780.0,352469.00,341091.00,336839.00,332004.00,325275.00,317786.00,346431.00,346449.00,342435.00,340007.00,351839.00,377976.00,387592.00,385726.00,372877.0,368219.00,361196.00,352531.00,340446.00,377154.00,373290.00,370869.00,382265.00,409815.00,421483.00,427419.00,391526.00,385219.00,376822.00,366674.00,356807.00,391340.00,386866.00,382997.00,393785.00,421228.00,432289.000000,431907.000000,424714.000000,420982.00,414941.000000,406608.000000,397041.000000,23992.000000,23881.000000,23382.000000,23906.000000,25171.000000,25522.00,26231.000000,26137.000000,26241.000000,31369.000000,31922.000000,32047.000000,411385.00000,407020.000000,404156.000000,413680.000000,438962.000000,448253.00,476830.000000,474702.000000,470317.00,464893.000000,456239.000000,446026.000000
164,Яйца,ПАВЛОДАРСКАЯ ОБЛАСТЬ,9637.30,8361.40,9164.50,10447.60,14083.60,14200.20,14200.20,12827.60,15190.30,11469.40,10976.10,9622.10,8446.10,8446.10,8895.50,10558.10,11831.20,13814.2,14183.70,14087.90,16244.10,12141.90,10834.60,9718.40,8614.40,10405.3,12266.80,13336.2,14884.70,16037.00,16325.40,16077.70,16598.20,15195.10,15267.20,15295.50,13864.00,13348.20,15943.10,18282.40,19924.00,19757.80,19774.80,20310.00,20590.20,18213.50,16278.30,18184.2,17131.70,16581.0,19203.20,20499.90,21971.0,22429.20,22393.20,22997.00,23738.50,21722.60,20525.60,18886.20,18343.50,18280.50,20271.50,20828.10,21491.70,22670.40,24237.50,24123.6,23045.90,20280.20,20677.80,19639.50,19152.10,17237.40,19001.20,20881.80,23706.20,23868.10,25091.30,24541.00,24043.40,20384.00,18617.30,17884.40,18605.20,16439.30,18710.20,20901.70,20292.20,21162.100000,22709.000000,23348.600000,21412.10,18511.500000,17998.400000,17788.500000,16233.800000,15161.000000,17138.800000,17299.100000,17224.400000,18089.10,18809.100000,18631.100000,17835.300000,16596.800000,14092.500000,15766.300000,16000.70000,15324.700000,16631.100000,16747.100000,17414.000000,18832.40,18654.700000,18689.800000,18071.90,16535.500000,15899.200000,14834.900000
102,Овцы и козы,РЕСПУБЛИКА КАЗАХСТАН,17429.73,15965.57,25342.99,21907.05,20597.75,37409.49,24157.41,27474.67,34201.49,28335.67,31345.02,49128.27,17505.24,15909.58,25960.76,22396.92,21324.33,38967.2,23554.97,27320.78,34858.36,27259.67,32464.04,54082.25,17781.61,16102.6,26367.01,21780.9,21173.74,38877.01,22469.33,26475.66,33547.06,26899.81,33408.02,61038.34,17920.05,15816.03,26885.76,21997.47,21127.73,40641.78,22644.33,26253.43,31914.82,26421.83,32052.33,59985.7,17776.61,15997.8,27311.62,22344.01,21754.7,39316.14,22723.67,26344.93,32568.02,26744.42,31772.48,59269.73,17948.56,16054.72,27686.04,22027.72,21214.93,39515.21,22976.22,26331.5,33391.95,27178.08,31954.38,60368.12,18458.38,16565.42,28322.34,22478.34,21763.05,40520.23

In [22]:
# save into an excel file: все 8 показателей в сумме по категориям хозяйств в одном excel
df_combined.to_excel("output data/Просуммированные по категориям с доп регрессорами.xlsx", index=False)

In [ ]:
# # save into excel files: 8 показателей в сумме по категориям хозяйств, каждый в отдельном excel

# indicators = ["КРС", "Лошади", "Свиньи", "Овцы и козы", "Птицы", "Молоко", "Яйца", "Верблюды"]

# for indicator in indicators:
#     df_filtered = df_combined[df_combined["Показатель"] == indicator].drop(columns=["Показатель"])
#     file_name = f"output data/{indicator}_обработанные.xlsx"
#     df_filtered.to_excel(file_name, index=False)
